# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [3]:
# Drop unnecessary columns
new_df = df.drop('timestamp', axis=1)


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [9]:
from surprise import Reader, Dataset
# read in values as Surprise dataset

reader = Reader()

data = Dataset.load_from_df(new_df, reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [10]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [11]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [44]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
param_grid = {'n_factors': [20,50, 100], 'reg_all': [.02, .05, .1]}

gs_model = GridSearchCV(SVD, param_grid=param_grid, n_jobs=-1, joblib_verbose=5)
gs_model.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  40 out of  45 | elapsed:   24.3s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   24.9s finished


In [45]:
# print out optimal parameters for SVD after GridSearch
print(gs_model.best_score)
print(gs_model.best_params)

{'rmse': 0.8687321229905768, 'mae': 0.6680362744919408}
{'rmse': {'n_factors': 50, 'reg_all': 0.05}, 'mae': {'n_factors': 50, 'reg_all': 0.05}}


In [46]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name': 'pearson', 'user_based': True})
knn_basic_cross_val = cross_validate(knn_basic, data, n_jobs=-1)

In [47]:
# print out the average RMSE score for the test set
for i in knn_basic_cross_val.items():
    print(i)
print(' ')
print(np.mean(knn_basic_cross_val['test_rmse']))

('test_rmse', array([0.9770714 , 0.97085418, 0.97059201, 0.97679511, 0.96353099]))
('test_mae', array([0.75619312, 0.74770506, 0.7505556 , 0.75157555, 0.74770402]))
('fit_time', (0.31389808654785156, 0.3210740089416504, 0.3211658000946045, 0.3027610778808594, 0.28940391540527344))
('test_time', (1.0492286682128906, 1.0506579875946045, 1.042480230331421, 1.0307328701019287, 1.0185420513153076))
 
0.9717687395481427


In [48]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name': 'pearson', 'user_base': True})
knn_baseline_cross_val = cross_validate(knn_baseline, data, n_jobs=-1)

In [49]:
# print out the average score for the test set
for i in knn_baseline_cross_val.items():
    print(i)
print(' ')
print(np.mean(knn_baseline_cross_val['test_rmse']))

('test_rmse', array([0.87711812, 0.88080374, 0.86851461, 0.8794243 , 0.87392682]))
('test_mae', array([0.67064945, 0.67063738, 0.6659532 , 0.67069822, 0.66864245]))
('fit_time', (0.39435482025146484, 0.40475988388061523, 0.4088258743286133, 0.3645198345184326, 0.34676504135131836))
('test_time', (1.4295151233673096, 1.448639154434204, 1.4512860774993896, 1.4612963199615479, 1.422234058380127))
 
0.8759575157863807


Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [89]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [90]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [50]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [51]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.047276554329698, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [67]:
rand_num = np.random.randint(0, len(df_movies))
rand_num

8821

In [81]:
def movie_rater(movie_df, num, genre=None):
    user_id = 1000
    rating_list = []
    rating_total = 0

    while len(rating_list) < 5:
        if genre == None:
            film = movie_df.iloc[np.random.randint(0, len(df_movies))]
        else:
            genre_df = movie_df[movie_df['genres'] in genre]
            film = genre_df.iloc[np.random.randint(0, len(df_movies))]
        
        rating = input(f"How would you rate {film['title']} on a scale of 1 to 5? \
        Press n if you have not seen it: ")
        
        if rating in ['1', '2', '3', '4', '5']:
            rating_total += 1
            rating_list.append({'userId': user_id, 'movieId': film['movieId'], 'rating': int(rating)})
        elif rating == 'n':
            continue
    
    return rating_list

In [83]:
# try out the new function here!
user_rating = movie_rater(df_movies, 5)

How would you rate Cat People (1982) on a scale of 1 to 5?         Press n if you have not seen it: 5
How would you rate Savage Nights (Nuits fauves, Les) (1992) on a scale of 1 to 5?         Press n if you have not seen it: 3
How would you rate Blue Steel (1990) on a scale of 1 to 5?         Press n if you have not seen it: 4
How would you rate Return of Martin Guerre, The (Retour de Martin Guerre, Le) (1982) on a scale of 1 to 5?         Press n if you have not seen it: 3
How would you rate Secret of Kells, The (2009) on a scale of 1 to 5?         Press n if you have not seen it: 2


If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [38]:
#user_rating

[{'userId': 1000, 'movieId': 55245, 'rating': '5'},
 {'userId': 1000, 'movieId': 2491, 'rating': '4'},
 {'userId': 1000, 'movieId': 4718, 'rating': '4'},
 {'userId': 1000, 'movieId': 5990, 'rating': '3'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [91]:
## add the new ratings to the original ratings DataFrame
new_ratings_df = new_df.append(user_rating, ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df, reader)

In [92]:
# train a model using the new combined DataFrame
movie_svd = SVD(n_factors=50, reg_all=.05)

movie_svd.fit(new_data.build_full_trainset())

In [93]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
movie_list = []
for movie in new_df['movieId'].unique():
    movie_list.append( (movie, movie_svd.predict(1000, movie)[3]))

In [97]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(movie_list, key=lambda x: x[1], reverse=True)
ranked_movies

[(898, 4.373870006493745),
 (1204, 4.337937494937032),
 (1237, 4.322655371006134),
 (1262, 4.307084183283736),
 (318, 4.296422799405778),
 (1208, 4.28770667684179),
 (904, 4.276868837122336),
 (750, 4.274790416745744),
 (858, 4.273280779248003),
 (1104, 4.270424971556268),
 (4973, 4.2632084171303495),
 (1233, 4.261559895877709),
 (1225, 4.240757558681002),
 (48516, 4.239039688369467),
 (1197, 4.223245102203662),
 (720, 4.217197212033273),
 (38061, 4.214782229874309),
 (1223, 4.208874944001206),
 (58559, 4.208520997007417),
 (50, 4.206794241005506),
 (1283, 4.206578476726323),
 (1203, 4.1999761250428405),
 (1089, 4.197513521150183),
 (168252, 4.196725241561768),
 (2160, 4.195676866449981),
 (951, 4.191917305477216),
 (7361, 4.190702019233487),
 (866, 4.190289708186291),
 (247, 4.185718850791065),
 (44195, 4.184908752058774),
 (1266, 4.183144578004674),
 (2329, 4.179668154179305),
 (3451, 4.176577405927353),
 (1252, 4.174228626151907),
 (912, 4.17410627945879),
 (142488, 4.17306367932948

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [107]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        user_ratings_sorted = sorted(user_ratings, key=lambda x: x[1], reverse=True)
        rec_list = []
        
        for movie in range(0, n):
            m_id = user_ratings_sorted[movie][0]
            movie_info = movie_title_df[movie_title_df['movieId'] == m_id]
            title = movie_info['title']
            rec_list.append(title)
        
        for i in range(0, len(rec_list)):
            print(f'Recommendation #{i+1}: {rec_list[i]}')
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation #1: 680    Philadelphia Story, The (1940)
Name: title, dtype: object
Recommendation #2: 906    Lawrence of Arabia (1962)
Name: title, dtype: object
Recommendation #3: 937    Seventh Seal, The (Sjunde inseglet, Det) (1957)
Name: title, dtype: object
Recommendation #4: 961    Great Escape, The (1963)
Name: title, dtype: object
Recommendation #5: 277    Shawshank Redemption, The (1994)
Name: title, dtype: object


## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.